In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import pickle

import pandas as pd

mentions = pickle.load(open("../data/mentions texts.pickle", "rb"))
mentions.drop(columns=["messageid"], inplace=True)
mentions.drop(index=mentions[mentions.MessageText.str.len() == 0].index, inplace=True)
mentions.drop(index=mentions[mentions.issuerid < 0].index, inplace=True)
sentiment = pickle.load(open("../data/sentiment_texts.pickle", "rb"))
sentiment.drop(
    index=sentiment[sentiment.MessageText.str.len() == 0].index, inplace=True
)


In [8]:
sentiment.drop(
    index=sentiment[sentiment.SentimentScore == 0].index, inplace=True
)

In [9]:
sentiment

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [10]:
print(len(mentions), len(sentiment))

ment = mentions[["issuerid", "MessageText"]]
sent = sentiment[["issuerid", "MessageText", "SentimentScore"]]

mrg = pd.merge(ment, sent, on=["issuerid", "MessageText"], how="right")

19353 9125


In [11]:
def apply_dataset_simple(group: pd.DataFrame):
    return ";".join(map(str, group["issuerid"].tolist()))


def apply_dataset_hard(group: pd.DataFrame):
    issuer = group["issuerid"].tolist()
    score = group["SentimentScore"].tolist()
    return ";".join([f"{i}-{s}" for i, s in zip(issuer, score)])


messages = []
labels = []
for message, group in mentions.groupby("MessageText"):
    messages.append(message)
    labels.append(apply_dataset_simple(group))

dataset = pd.DataFrame({"message": messages, "label": labels})
dataset.to_csv("../data/data2.csv", index=False)

In [12]:
messages = []
labels = []
for message, group in mrg.groupby("MessageText"):
    messages.append(message)
    labels.append(apply_dataset_hard(group))

dataset = pd.DataFrame({"message": messages, "label": labels})
dataset.to_csv("../data/data-hard2.csv", index=False)